# exiobase to isic v4 

this correspondance table does not exist but here we create it from existing ones. 

In [1]:
import pandas as pd

from https://unstats.un.org/unsd/classifications/business-trade/tables/ISIC/ISIC31_ISIC4/ISIC31_ISIC4.txt

In [2]:
isic31to4=pd.read_table('../raw/ISIC31_ISIC4.txt',sep=',')
isic31to4.head()

C:\Users\migue\Anaconda3\envs\bw2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,ISIC31code,partialISIC31,ISIC4code,partialISIC4,Detail
0,111,1,111,0,NaN
1,111,1,112,0,NaN
2,111,1,113,1,"Growing of sugar beet, roots and tubers"
3,111,1,114,0,NaN
4,111,1,115,0,NaN


In [3]:
isic_31_to_4_dict=dict(isic31to4[['ISIC31code','ISIC4code']].values)

provided by Constantin Standler

In [4]:
exiobase_to_isic3=pd.read_excel('../raw/ISIC REV. 3 - EXIOBASE2.0.xlsx',usecols='B:C')
exiobase_to_isic3.head()

,ISIC REV. 3 - NACE REV. 1,EXIOBASE 2.0
0,Source (unique),NaN
1,1,i01.a;i01.b;i01.c;i01.d;i01.e;i01.f;i01.g;i01....
2,11,i01.a;i01.b;i01.c;i01.d;i01.e;i01.f;i01.g;i01.h
3,111,i01.a;i01.b;i01.c;i01.e;i01.f;i01.g;i01.h
4,112,i01.d;i01.e;i01.h


ok, this needs further processing because codes are given at different levels of aggregation.

In [5]:
exiobase_to_isic3=exiobase_to_isic3.drop(0)

In [6]:
exiobase_to_isic3.columns=['isic3','exiobase2']

In [7]:
exiobase_to_isic3.isic3=exiobase_to_isic3.isic3.astype(str)

if the isic code has 4 digits then is level 4

In [8]:
exiobase_to_isic3_l4=exiobase_to_isic3[exiobase_to_isic3.isic3.str.match('....')]

In [9]:
exiobase_to_isic3_l4.head(5)

,isic3,exiobase2
23,1010,i10
25,1020,i10
27,1030,i10
29,1110,i11.a;i11.b;i11.c
30,1120,i11.a;i11.b;i11.c


we can iterate the columns and parse the string in exiobase separating by `;`. Create a dictionary of the exiobase codes to isic codes.

In [10]:
ex2_isic3_l4={}
for row in exiobase_to_isic3_l4.iterrows():
    for excode in row[1]['exiobase2'].split(';'):
        excode=excode.strip()
        ex2_isic3_l4[excode]=row[1]['isic3']

In [11]:
len(ex2_isic3_l4)

143

now isic3 to isic 4

In [12]:
ex2_isic4_l4={}
for item in ex2_isic3_l4.items():
    ex2_isic4_l4[item[0]]=str(isic_31_to_4_dict[int(item[1])])

In [13]:
len(ex2_isic4_l4)

143

now from exiobase 2 to 3

identify the missing codes

In [38]:
ex3=pd.read_excel('../raw/Classifications_v_3_3_17.xlsx',sheet_name='Activities',index_col=[0,1])
ex3.head(3)

Activity code 1 Activity code 2
Contry code Activity name                                                   
AU          Cultivation of paddy rice                  i01.a          A_PARI
            Cultivation of wheat                       i01.b          A_WHEA
            Cultivation of cereal grains nec           i01.c          A_OCER

In [47]:
(ex3.groupby('Contry code').count()==164).all()

Activity code 1    True
Activity code 2    True
dtype: bool

All the countries have the same number of activities, so we can pick one to identify the codes. 

In [53]:
ex3_descrip_to_actcode1=dict(ex3.loc['AU','Activity code 1'])

In [54]:
ex3_actcode1_to_descrip = {v: k for k, v in ex3_descrip_to_actcode1.items()}

In [31]:
ex3_actcode1_set=set(ex3['Activity code 1'].values)

In [33]:
missing_codes=ex3_actcode1_set.difference(set([k for k in ex2_isic4_l4.keys()]))

In [61]:
for code in missing_codes:
    print(code,'-',ex3_actcode1_to_descrip[code])

i90.5.d - Landfill of waste: Inert/metal/hazardous
i01.f - Cultivation of sugar cane, sugar beet
i24.a - Plastics, basic
i01.a - Cultivation of paddy rice
i90.2.a - Biogasification of food waste, incl. land application
i05 - Fishing, operating of fish hatcheries and fish farms; service activities incidental to fishing (05)
i01.k - Poultry farming
i01.d - Cultivation of vegetables, fruit, nuts
i90.5.c - Landfill of waste: Plastic
i90.2.c - Biogasification of sewage slugde, incl. land application
i24.a.w - Re-processing of secondary plastic into new plastic
i01.o - Wool, silk-worm cocoons
i01.b - Cultivation of wheat
i01.w.1 - Manure treatment (conventional), storage and land application
i90.2.b - Biogasification of paper, incl. land application
i26.a.w - Re-processing of secondary glass into new glass
i01.e - Cultivation of oil seeds
i90.5.f - Landfill of waste: Wood
i01.g - Cultivation of plant-based fibers
i26.d.w - Re-processing of ash into clinker
i01.c - Cultivation of cereal grain

In [57]:
#assume they have not changed.
ex3_isic4_l4=ex2_isic4_l4.copy()

In [58]:
ex3_isic4_l4['i90.5.d']='5093' 
ex3_isic4_l4['i01.f']='0133'
ex3_isic4_l4['i24.a']=
ex3_isic4_l4['i01.a']=
ex3_isic4_l4['i90.2.a']=
ex3_isic4_l4['i05']=
ex3_isic4_l4['i01.k']=
ex3_isic4_l4['i01.d']=
ex3_isic4_l4['i90.5.c']=
ex3_isic4_l4['i90.2.c']=
ex3_isic4_l4['i01.o']=
ex3_isic4_l4['i01.b']=
ex3_isic4_l4['i90.2.c']=
ex3_isic4_l4['i90.2.c']=
ex3_isic4_l4['i90.2.c']=
ex3_isic4_l4['i90.2.c']=
ex3_isic4_l4['i90.2.c']=


{'i10': '990',
 'i11.a': '910',
 'i11.b': '910',
 'i11.c': '910',
 'i12': '990',
 'i13.1': '990',
 'i13.20.11': '990',
 'i13.20.12': '990',
 'i13.20.13': '990',
 'i13.20.14': '990',
 'i13.20.15': '990',
 'i13.20.16': '990',
 'i14.1': '990',
 'i14.2': '990',
 'i14.3': '990',
 'i15.a': '1010',
 'i15.b': '1010',
 'i15.c': '1010',
 'i15.d': '1010',
 'i15.k': '1075',
 'i15.g': '1075',
 'i15.i': '3530',
 'i15.e': '1040',
 'i15.f': '1050',
 'i15.h': '1072',
 'i15.j': '1104',
 'i16': '1200',
 'i17': '1430',
 'i18': '1511',
 'i19': '3230',
 'i20': '3319',
 'i20.w': '3319',
 'i21.1': '1701',
 'i21.w.1': '2220',
 'i21.2': '2220',
 'i22': '1820',
 'i23.1': '1910',
 'i23.2': '1920',
 'i23.3': '3822',
 'i24.1': '2013',
 'i24.1.w': '2013',
 'i24.4': '2030',
 'i24.2': '3821',
 'i24.3': '3821',
 'i25': '3319',
 'i26.a': '3319',
 'i26.w.1': '3319',
 'i26.b': '2391',
 'i26.c': '2392',
 'i26.d': '2394',
 'i26.e': '3319',
 'i27.a': '2431',
 'i27.a.w': '2431',
 'i27.41': '2420',
 'i27.41.w': '2420',
 'i27.4

In [14]:
ex2_isic4_l4_df=pd.DataFrame.from_dict(ex2_isic4_l4,orient='index').reset_index()
ex2_isic4_l4_df.columns=['exiobase2','isic_rev4_l4']
ex2_isic4_l4_df.head(3)

,exiobase2,isic_rev4_l4
0,i10,990
1,i11.a,910
2,i11.b,910


In [106]:
ex2_isic4_l4_df.to_csv('../raw/exiobase2_isic_rev4_l4.csv',index=False)

# option B) use NACE codes

In [ ]:
pd.read_csv